In [ ]:
pip install ultralytics opencv-python numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 44.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [ ]:
pip install ultralytics opencv-python numpy filterpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=6114982b05b5bdb7141fa4346bf20272731bb7bb2d58f65bf8a5185181c4f4b3
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [ ]:
# model = YOLO('/content/drive/MyDrive/best.pt')
cap_x = cv2.VideoCapture('/content/drive/MyDrive/broadcast.mp4')
cap_y = cv2.VideoCapture('/content/drive/MyDrive/tacticam.mp4')

In [ ]:
# Download sort.py
!wget https://raw.githubusercontent.com/abewley/sort/master/sort.py

import cv2
import numpy as np
from ultralytics import YOLO
from sort import Sort

# Load your YOLOv11 model
model = YOLO('/content/drive/MyDrive/best.pt')

def get_player_detections(frame):
    results = model(frame)
    detections = []
    for r in results[0].boxes:
        cls_id = int(r.cls[0])
        if cls_id == 2:  # Adjust if your 'player' class index is different
            x1, y1, x2, y2 = map(int, r.xyxy[0])
            conf = float(r.conf[0])
            detections.append([x1, y1, x2, y2, conf])
    return np.array(detections) if detections else np.empty((0, 5))

def get_histogram(frame, bbox):
    x1, y1, x2, y2 = map(int, bbox)
    crop = frame[y1:y2, x1:x2]
    if crop.size == 0:
        return np.zeros((512,))
    hist = cv2.calcHist([crop], [0, 1, 2], None, [8, 8, 8],
                        [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

# --- Step 1: Process Video X, assign IDs, save features ---
tracker_x = Sort(max_age=30, min_hits=1, iou_threshold=0.3)
features_db = {}  # {id: [histogram]}


cap_x = cv2.VideoCapture('/content/drive/MyDrive/broadcast.mp4')
width = int(cap_x.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap_x.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap_x.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_x = cv2.VideoWriter('x_output_corrected.mp4', fourcc, fps, (width, height))

while True:
    ret, frame = cap_x.read()
    if not ret:
        break
    detections = get_player_detections(frame)
    tracks = tracker_x.update(detections)
    for track in tracks:
        x1, y1, x2, y2, track_id = map(int, track)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        # Save/update feature for this ID
        features_db[track_id] = get_histogram(frame, (x1, y1, x2, y2))
    out_x.write(frame)
cap_x.release()
out_x.release()

# --- Step 2: Process Video Y, assign IDs based on X ---
cap_y = cv2.VideoCapture('/content/drive/MyDrive/tacticam.mp4')
out_y = cv2.VideoWriter('y_output_corrected.mp4', fourcc, fps, (width, height))
next_id = max(features_db.keys()) + 1 if features_db else 1

def match_hist(hist, features_db, threshold=0.5):
    best_id = None
    best_score = float('inf')
    for id_, ref_hist in features_db.items():
        score = cv2.compareHist(hist, ref_hist, cv2.HISTCMP_BHATTACHARYYA)
        if score < best_score:
            best_score = score
            best_id = id_
    if best_score < threshold:
        return best_id
    return None

while True:
    ret, frame = cap_y.read()
    if not ret:
        print("why4")
        break
    detections = get_player_detections(frame, model)
    ids_y = []
    for det in detections:
        x1, y1, x2, y2, conf = map(int, det)
        hist = get_histogram(frame, (x1, y1, x2, y2))
        center = get_center((x1, y1, x2, y2))
        # Compare with all X players using visual and temporal features
        best_id = None
        best_score = float('inf')
        for id_, feat in features_db.items():
            visual_dist = cv2.compareHist(hist, feat['hist'], cv2.HISTCMP_BHATTACHARYYA)
            traj_y = [center] * min(10, len(feat['traj']))
            traj_x = feat['traj'][-len(traj_y):]
            temp_dist = trajectory_similarity(traj_y, traj_x)
            print("why5")
            score = 0.7 * visual_dist + 0.3 * (temp_dist / max(width, height))
            if score < best_score:
                best_score = score
                best_id = id_
        # Only assign ID if match is good enough
        if best_score < 0.7:  # Tune threshold as needed
            ids_y.append(best_id)
            label = f'ID: {best_id}'
        else:
            ids_y.append(-1)  # No new ID
            label = 'ID: ?'
        # Draw
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    out_y.write(frame)
cap_y.release()
out_y.release()

--2025-06-23 13:49:29--  https://raw.githubusercontent.com/abewley/sort/master/sort.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11739 (11K) [text/plain]
Saving to: ‘sort.py.3’

sort.py.3           100%[===================>]  11.46K  --.-KB/s    in 0s      

2025-06-23 13:49:29 (26.8 MB/s) - ‘sort.py.3’ saved [11739/11739]


0: 384x640 3 players, 2667.8ms
Speed: 4.9ms preprocess, 2667.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 2474.3ms
Speed: 4.9ms preprocess, 2474.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 players, 2461.2ms
Speed: 4.6ms preprocess, 2461.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 players, 2434.2ms
Speed: 4.4m

TypeError: get_player_detections() takes 1 positional argument but 2 were given

In [ ]:
# --- Step 2: Process Video Y, assign IDs based on X ---
cap_y = cv2.VideoCapture('/content/drive/MyDrive/tacticam.mp4')
out_y = cv2.VideoWriter('y_output_corrected.mp4', fourcc, fps, (width, height))
next_id = max(features_db.keys()) + 1 if features_db else 1

def match_hist(hist, features_db, threshold=0.5):
    best_id = None
    best_score = float('inf')
    for id_, ref_hist in features_db.items():
        score = cv2.compareHist(hist, ref_hist, cv2.HISTCMP_BHATTACHARYYA)
        if score < best_score:
            best_score = score
            best_id = id_
    if best_score < threshold:
        return best_id
    return None

# Need to define get_center and trajectory_similarity functions
def get_center(bbox):
    x1, y1, x2, y2 = bbox
    return ((x1 + x2) // 2, (y1 + y2) // 2)

def trajectory_similarity(traj1, traj2):
    # Simple Euclidean distance between the last points as a placeholder
    # You might need a more sophisticated trajectory comparison
    if not traj1 or not traj2:
        return float('inf') # Cannot compare empty trajectories
    p1 = np.array(traj1[-1])
    p2 = np.array(traj2[-1])
    return np.linalg.norm(p1 - p2)


while True:
    ret, frame = cap_y.read()
    if not ret:
        print("why4")
        break
    # Removed the extra 'model' argument
    detections = get_player_detections(frame)
    ids_y = []
    for det in detections:
        x1, y1, x2, y2, conf = map(int, det)
        hist = get_histogram(frame, (x1, y1, x2, y2))
        center = get_center((x1, y1, x2, y2))
        # Compare with all X players using visual and temporal features
        best_id = None
        best_score = float('inf')
        for id_, feat in features_db.items():
            # Check if 'hist' and 'traj' keys exist in feat
            if 'hist' in feat and 'traj' in feat:
                visual_dist = cv2.compareHist(hist, feat['hist'], cv2.HISTCMP_BHATTACHARYYA)
                traj_y = [center] * min(10, len(feat['traj']))
                traj_x = feat['traj'][-len(traj_y):]
                temp_dist = trajectory_similarity(traj_y, traj_x)
                print("why5")
                score = 0.7 * visual_dist + 0.3 * (temp_dist / max(width, height))
                if score < best_score:
                    best_score = score
                    best_id = id_
        # Only assign ID if match is good enough
        if best_score < 0.7:  # Tune threshold as needed
            ids_y.append(best_id)
            label = f'ID: {best_id}'
        else:
            ids_y.append(-1)  # No new ID
            label = 'ID: ?'
        # Draw
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    out_y.write(frame)
cap_y.release()
out_y.release()


0: 384x640 22 players, 2 referees, 2438.2ms
Speed: 7.9ms preprocess, 2438.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 1 referee, 2452.8ms
Speed: 4.2ms preprocess, 2452.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 1 referee, 2466.0ms
Speed: 3.6ms preprocess, 2466.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 3286.4ms
Speed: 5.1ms preprocess, 3286.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 22 players, 1 referee, 3232.3ms
Speed: 4.7ms preprocess, 3232.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 1 referee, 2466.9ms
Speed: 4.1ms preprocess, 2466.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 players, 1 referee, 2411.5ms
Speed: 3.3ms preprocess, 2411.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

